In [1]:
import pandas as pd
import smtplib
import os
import json
import matplotlib.pyplot as plt
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email import encoders
from email.utils import make_msgid

In [2]:
# Configuración de comportamiento
ORIGEN = 'misiontic.noresponder@outlook.com'
ASUNTO = 'GRUPOS Y HORARIOS ASIGNADOS - CICLO 2'
SERVIDOR_SMTP = 'smtp-mail.outlook.com:587'
CORREO_TEST = False
ENVIO_A_PENDIENTES = False

if CORREO_TEST:
    RUTA = './INPUT/INFO_CURSOS/TEST_3.xlsx'
elif ENVIO_A_PENDIENTES:
    RUTA = './INPUT/INFO_CURSOS/ENVIOS_PENDIENTES_CREDENCIALES.xlsx'
else:
    RUTA = './INPUT/INFO_CURSOS/LISTA_CREDENCIALES_FORMADORES.xlsx'

COLUMNA_CORREO = 'CORREO_DOCENTE'
COLUMNAS = [
    'ASIGNATURA',
    'GRUPO',
    'DIA',
    'HORA_INICIAL_',
    'HORA_FINAL_',
    'CUENTA_ZOOM_UIS_DEFINITIVA_2022',    
    'CONTRASEÑA_ZOOM',
    'ENLACE_ZOOM_CLASES_SINCRÓNICAS',
]

In [3]:
MSG_HEAD = """<html><body><p>Bucaramanga, 11 de junio de 2022<br>
Respetado profesional<br>
<b>{formador}</b><br>
Programa Misión TIC 2022 - Ciclo 2</p>

"""
MSG_BODY = """<p>Dado que este lunes 13 de junio inicia el ciclo 2, se le comparten sus grupos asignados y sus horarios:<br></p>

"""

MSG_FOOTER = """<p>ESTE CORREO SE ENVIÓ DE FORMA AUTOMÁTICA, NO RESPONDA A ESTA DIRECCIÓN O CORREO. Cualquier información, contacte a misiontic.monitor@uis.edu.co o rectoria.misiontic@uis.edu.co.</p>
<p>Tecnología diseñada por <a href="https://nuwebs.com.co">Nuwebs</a></p>
<p>Bucaramanga, Colombia. +57 3184301032</p>
</body></html>"""

In [4]:
f = open('CREDENCIALES.json')
CREDENCIALES = json.load(f)
f.close()
datos = pd.read_excel(RUTA, engine = 'openpyxl')

In [6]:
datos

,ASIGNATURA,GRUPO,DIA,HORA_INICIAL_,HORA_FINAL_,DOCENTE,NOMBRE_DOCENTE,CORREO_DOCENTE,CORREO_INSTITUCIONAL_DOCENTE,TELEFONO_DOCENTE,CUENTA_ZOOM_UIS_DEFINITIVA_2022,ENLACE_ZOOM_CLASES_SINCRÓNICAS,CONTRASEÑA_ZOOM
0,JAVA,H54,LUNES A VIERNES,20:00:00,22:00:00,73126121,ROY HERNANDO LLAMAS MUNOZ,rhllamas@gmail.com,misiontic.formador58@uis.edu.co,3233985677,zoom831@uis.edu.co,https://renata.zoom.us/j/81372682151,5Cr4qYrjD1S
1,JAVA,H59,LUNES A VIERNES,20:00:00,22:00:00,1049413439,DARIO ALEJANDRO RIANO VELANDIA,alejandroria.94@gmail.com,misiontic.formador8@uis.edu.co,3144416477,zoom806@uis.edu.co,https://renata.zoom.us/j/83500201712,3$n2V$7$DS
2,JAVA,H28,LUNES A VIERNES,16:00:00,18:00:00,91473712,ROGERIO ORLANDO BELTRAN CASTRO,rogeriobeltran@gmail.com,misiontic.formador30@uis.edu.co,3183867641,zoom816@uis.edu.co,https://renata.zoom.us/j/87627259732,67T$b2$cds
3,JAVA,H13,LUNES A VIERNES,8:00:00,10:00:00,13513031,GUILLERMO BEJARANO REYES,gbejarano@gmail.com,misiontic.formador73@uis.edu.co,3152910310 - 76383090,zoom844@uis.edu.co,https://renata.zoom.us/j/83740582238,7Zqx1vm1DS
4,JAVA,H12,LUNES A VIERNES,8:00:00,10:00:00,91254478,SERGIO ARTURO MEDINA CASTILLO,smedina_castillo@yahoo.es,misiontic.formador48@uis.edu.co,3132086234,zoom834@uis.edu.co,https://renata.zoom.us/j/87162679458,58U5sS22DS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,COACHING II,N44,Martes,17:00:00,18:00:00,63553366,ANA VICTORIA VARGAS PIMIENTO,axlvictoria@gmail.com,misiontic.coach8@uis.edu.co,3108590814,zoom872@uis.edu.co,https://renata.zoom.us/j/89665734424,78Mwc5@MDS
203,COACHING II,N12,Miércoles,8:00:00,9:00:00,1065880115,VIVIAN CAROLINA PEREZ ALMANZA,ps.vivianperez@gmail.com,misiontic.coach6@uis.edu.co,3133929166,zoom828@uis.edu.co,https://renata.zoom.us/j/89586640780,4qN3Q6t7DS
204,COACHING II,N71,Jueves,21:00:00,22:00:00,1065880115,VIVIAN CAROLINA PEREZ ALMANZA,ps.vivianperez@gmail.com,misiontic.coach6@uis.edu.co,3133929166,zoom828@uis.edu.co,https://renata.zoom.us/j/89586640780,4qN3Q6t7DS
205,COACHING II,N47,Viernes,17:00:00,18:00:00,63553366,ANA VICTORIA VARGAS PIMIENTO,axlvictoria@gmail.com,misiontic.coach8@uis.edu.co,3108590814,zoom872@uis.edu.co,https://renata.zoom.us/j/89665734424,78Mwc5@MDS


In [7]:
def generarCorreo(origen, destino, asunto):
    multipart = MIMEMultipart('related') # Posible quitar related
    multipart['From'] = origen
    multipart['To'] = destino
    multipart['Subject'] = asunto
    multipart['X-Priority'] = '2'
    multipart.preamble = '====================================================='
    
    return multipart

def getCuerpoCorreo(multipart, formador, df): 
    msgAlternative = MIMEMultipart('alternative') #Posible quitar esto
    multipart.attach(msgAlternative)
    
    cuerpoAdicional = '<table border="1"><tbody><tr>'
    for columna in COLUMNAS:
        cuerpoAdicional += '<th>' + columna + '</th>'
    cuerpoAdicional += '</tr>'
    for index, row in df.iterrows():
        cuerpoAdicional += '<tr>'
        for columna in COLUMNAS:            
            cuerpoAdicional += '<td>' + row[columna] + '</td>'
        cuerpoAdicional += '</tr>'        
    cuerpoAdicional += '</tbody></table>'
    cuerpoAdicional += '<p><b>Nota:</b> Si no puede visualizar la tabla, sus credenciales son el archivo adjunto a este correo</p>'
    cuerpo = MSG_HEAD.format(formador = formador) + MSG_BODY + cuerpoAdicional + MSG_FOOTER
    cuerpo = MIMEText(cuerpo, 'html', 'utf-8')
    msgAlternative.attach(cuerpo)
    return multipart, cuerpo

def getAdjuntos(multipart, imgs = [], adjuntos = []):
    for img in imgs:
        multipart.attach(img)
    for adjunto in adjuntos:
        f = open(adjunto, 'rb')
        nFile = MIMEApplication(f.read(), 'vnd.ms-excel')
        f.close()
        encoders.encode_base64(nFile)
        nFile.add_header('Content-Disposition', 'attachment', filename=os.path.basename(adjunto))
        multipart.attach(nFile)
    return multipart

In [ ]:
enviados = datos.copy()
destinos = datos[COLUMNA_CORREO].unique()
servidor = smtplib.SMTP(SERVIDOR_SMTP)
servidor.starttls()
servidor.login(CREDENCIALES['USUARIO'], CREDENCIALES['PASS'])
cont = 1
for destino in destinos:
    dfD = datos[datos[COLUMNA_CORREO] == destino]
    nombreFormador = dfD['NOMBRE_DOCENTE'].iloc[0]
    adjuntos = []
    nombreArchivo = 'GRUPOS_ASIGNADOS_' + nombreFormador.replace(' ', '_')
    ruta = './Temp/' + nombreArchivo + '.xlsx'
    dfD[COLUMNAS].to_excel(ruta, index = False)
    adjuntos.append(ruta)
    
    base = generarCorreo(ORIGEN, destino, ASUNTO)
    base, contenido = getCuerpoCorreo(base, nombreFormador, dfD)
    base = getAdjuntos(base, [], adjuntos)
    try:        
        servidor.sendmail(ORIGEN, destino, base.as_string())
        enviados = enviados[enviados[COLUMNA_CORREO] != destino]
    except:
        enviados.to_excel('./INPUT/INFO_CURSOS/ENVIOS_PENDIENTES_CREDENCIALES.xlsx', index=False)
        print ('Ocurrió un error al enviar el correo a', destino)
        break
    print (cont, 'Enviado a', nombreFormador, destino)
    cont += 1
servidor.quit()